# Query Data with AWS Data Wrangler

**AWS Data Wrangler** is an open-source Python library that extends the power of the Pandas library to AWS connecting DataFrames and AWS data related services (Amazon Redshift, AWS Glue, Amazon Athena, Amazon EMR, Amazon QuickSight, etc).

* https://github.com/awslabs/aws-data-wrangler
* https://aws-data-wrangler.readthedocs.io

Built on top of other open-source projects like Pandas, Apache Arrow, Boto3, s3fs, SQLAlchemy, Psycopg2 and PyMySQL, it offers abstracted functions to execute usual ETL tasks like load/unload data from Data Lakes, Data Warehouses and Databases.

_Note that AWS Data Wrangler is simply a Python library that uses existing AWS Services.  AWS Data Wrangler is not a separate AWS Service.  You install AWS Data Wrangler through `pip install` as we will see next._

# _Pre-Requisite: Make Sure You Created an Athena Table for Both TSV and Parquet in Previous Notebooks_

In [1]:
%store -r ingest_create_athena_table_tsv_passed

In [2]:
try:
    ingest_create_athena_table_tsv_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [3]:
print(ingest_create_athena_table_tsv_passed)

True


In [4]:
if not ingest_create_athena_table_tsv_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [5]:
%store -r ingest_create_athena_table_parquet_passed

In [6]:
try:
    ingest_create_athena_table_parquet_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not convert into Parquet data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [7]:
print(ingest_create_athena_table_parquet_passed)

True


In [8]:
if not ingest_create_athena_table_parquet_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not convert into Parquet data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


# Setup

In [9]:
import sagemaker
import boto3

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [11]:
!pip install awswrangler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.3/381.3 kB 13.2 MB/s eta 0:00:00


In [12]:
import awswrangler as wr

# Query Parquet from S3 with Push-Down Filters

Read Apache Parquet file(s) from from a received S3 prefix or list of S3 objects paths.

The concept of Dataset goes beyond the simple idea of files and enable more complex features like partitioning and catalog integration (AWS Glue Catalog): 

_dataset (bool)_ – If True read a parquet dataset instead of simple file(s) loading all the related partitions as columns.

In [13]:
p_filter = lambda x: x["product_category"] == "Digital_Software"

In [14]:
path = "s3://{}/amazon-reviews-pds/parquet/".format(bucket)
df_parquet_results = wr.s3.read_parquet(
    path, columns=["star_rating", "product_category", "review_body"], partition_filter=p_filter, dataset=True
)
df_parquet_results.shape

(102084, 3)

In [15]:
df_parquet_results.head(5)

,star_rating,review_body,product_category
0,4,So far so good,Digital_Software
1,3,Needs a little more work.....,Digital_Software
2,1,Please cancel.,Digital_Software
3,5,Works as Expected!,Digital_Software
4,4,I've had Webroot for a few years. It expired a...,Digital_Software


# Query Parquet from S3 in Chunks

Batching (chunked argument) (Memory Friendly):

Will enable the function to return a Iterable of DataFrames instead of a regular DataFrame.

There are two batching strategies on Wrangler:
* If chunked=True, a new DataFrame will be returned for each file in your path/dataset.
* If chunked=INTEGER, Wrangler will iterate on the data by number of rows equal to the received INTEGER.

P.S. chunked=True if faster and uses less memory while chunked=INTEGER is more precise in number of rows for each Dataframe.

In [35]:
path = "s3://{}/amazon-reviews-pds/parquet/".format(bucket)
chunk_iter = wr.s3.read_parquet(
    path,
    # columns=["star_rating", "review_body", "product_category"],
    # filters=[("product_category", "=", "Digital_Software")],
    partition_filter=p_filter,
    dataset=True,
    chunked=True,
)

In [36]:
print(next(chunk_iter))

       marketplace customer_id       review_id  product_id product_parent  \
0               US    17747349  R2EI7QLPK4LF7U  B00U7LCE6A      106182406   
1               US    10956619  R1W5OMFK1Q3I3O  B00HRJMOM4      162269768   
2               US    13132245   RPZWSYWRP92GI  B00P31G9PQ      831433899   
3               US    35717248  R2WQWM04XHD9US  B00FGDEPDY      991059534   
4               US    17710652  R1WSPK2RA2PDEF  B00FZ0FK0U      574904556   
...            ...         ...             ...         ...            ...   
102079          US    41754720  R19OFJV91M7D8X  B000YMR61A      141393130   
102080          US    51669529  R1I6G894K5AGG5  B000YMR61A      141393130   
102081          US    24731012  R17OE43FFEP81I  B000YMR5X4      234295632   
102082          US    16049580  R15MGDDK63B52Z  B000YMR61A      141393130   
102083          US    46098046  R1GGJJA2R68033  B000YMNI2Q      847631772   

                                            product_title  star_rating  \
0

# Query the Glue Catalog (ie. Hive Metastore)
Get an iterator of tables.

In [29]:
database_name = "dsoaws"
table_name_tsv = "amazon_reviews_tsv"
table_name_parquet = "amazon_reviews_parquet"

In [30]:
for table in wr.catalog.get_tables(database="dsoaws"):
    print(table["Name"])

amazon_reviews_parquet
amazon_reviews_tsv


# Query from Athena
Execute any SQL query on AWS Athena and return the results as a Pandas DataFrame.  


In [31]:
%%time
df = wr.athena.read_sql_query(sql="SELECT * FROM {} LIMIT 5000".format(table_name_parquet), database=database_name)

CPU times: user 1.57 s, sys: 172 ms, total: 1.74 s
Wall time: 5.43 s


In [32]:
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,year,review_date,product_category
0,US,17747349,R2EI7QLPK4LF7U,B00U7LCE6A,106182406,CCleaner Free [Download],4,0,0,N,Y,Four Stars,So far so good,2015,2015-08-31,Digital_Software
1,US,10956619,R1W5OMFK1Q3I3O,B00HRJMOM4,162269768,ResumeMaker Professional Deluxe 18,3,0,0,N,Y,Three Stars,Needs a little more work.....,2015,2015-08-31,Digital_Software
2,US,13132245,RPZWSYWRP92GI,B00P31G9PQ,831433899,Amazon Drive Desktop [PC],1,1,2,N,Y,One Star,Please cancel.,2015,2015-08-31,Digital_Software
3,US,35717248,R2WQWM04XHD9US,B00FGDEPDY,991059534,Norton Internet Security 1 User 3 Licenses,5,0,0,N,Y,Works as Expected!,Works as Expected!,2015,2015-08-31,Digital_Software
4,US,17710652,R1WSPK2RA2PDEF,B00FZ0FK0U,574904556,SecureAnywhere Intermet Security Complete 5 De...,4,1,2,N,Y,Great antivirus. Worthless customer support,I've had Webroot for a few years. It expired a...,2015,2015-08-31,Digital_Software


# Query from Athena in Chunks
Retrieving in chunks can help reduce memory requirements.  

_This will take a few seconds._

In [33]:
%%time

chunk_iter = wr.athena.read_sql_query(
    sql="SELECT * FROM {} LIMIT 5000".format(table_name_parquet),
    database="{}".format(database_name),
    chunksize=64_000,  # 64 KB Chunks
)

CPU times: user 1.46 s, sys: 152 ms, total: 1.62 s
Wall time: 5.04 s


In [34]:
print(next(chunk_iter))

     marketplace customer_id       review_id  product_id product_parent  \
0             US    17747349  R2EI7QLPK4LF7U  B00U7LCE6A      106182406   
1             US    10956619  R1W5OMFK1Q3I3O  B00HRJMOM4      162269768   
2             US    13132245   RPZWSYWRP92GI  B00P31G9PQ      831433899   
3             US    15696503  R2HGGCCZSSNUCB  B00M9GTJLY      103182180   
4             US     9723928   REEE4LHSVPRV9  B00H9A60O4      608720080   
...          ...         ...             ...         ...            ...   
4995          US    21787063  R2BUPAT05NCY9D  B00MHZ6Z64      249773946   
4996          US    13354628  R1ATU0AI3F2L45  B00M9GTHS4      925385625   
4997          US    42139108  R2BK67PBBDMQMR  B00PG8FOSY      801860929   
4998          US    44158292   RW84VOK5LHPV1  B00B1TGDAU       12764369   
4999          US    36972714  R3LDM8C3IQJ5JV  B00OPED7NO      151420322   

                                          product_title  star_rating  \
0                          

# Release Resources

In [37]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}